<img align="right" width="400" src="https://drive.google.com/thumbnail?id=1rPeHEqFWHJcauZlU82a4hXM10TUjmHxM&sz=s4000" alt="FHNW Logo">


# Sentiment Classification with LLMs (of Doctor Reviews)

by Fabian Märki

## Summary
The aim of this notebook is to showcase how to build a prompt for sentiment classification using a LLM (of your choice). It also showcases that it can be tricky to come up with a "good" prompt even for such a simple task and that the runtime (and with this also CO2 emissions) increase drastically compared to a simple classifier (e.g. using SVM with TF-IDF).

This notebook contains assigments: <font color='red'>Questions are written in red.</font>

<a href="https://colab.research.google.com/github/markif/2025_HS_CAS_NLP_LAB_Notebooks/blob/master/06_a_Sentiment_Classification_of_Doctor_Reviews_with_LLMs.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [14]:
%%capture

!pip install 'fhnw-nlp-utils>=0.11.0,<0.12.0'

In [2]:
#OLLAMA_HOST='https://XYZ.trycloudflare.com'
OLLAMA_HOST='http://localhost:11434'

# try the smallest model (let's hope for some speedup and that it is sufficient for this simple task)
MODEL_TEXT = "qwen3:0.6b"

# ensure (in your environment) that the model is installed
#
# !ollama pull qwen3:0.6b
# docker exec -it ollama sh -c "ollama pull qwen3:0.6b"

Let's use Ollama's client this time.

In [3]:
%%capture

!pip install ollama

In [4]:
from ollama import Client

client = Client(
    host = OLLAMA_HOST
)

In [5]:
def predict_sensitivity_en(text):
    
    return client.chat(
        model = MODEL_TEXT, 
        messages = [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": f"What is the sentiment of the following review: '{text}'"
            }
        ])

In [6]:
%%time

text_pos = "Der Arzt war sehr freundlich und kompetent. Die Wartezeit war kurz."

response_pos = predict_sensitivity_en(text_pos)
print(response_pos, "\n\n")

text_neg = "Der Arzt war sehr unfreundlich und unkompetent. Die Wartezeit war lang."

response_neg = predict_sensitivity_en(text_neg)
print(response_neg, "\n\n")

model='qwen3:0.6b' created_at='2025-05-30T12:35:10.932949063Z' done=True done_reason='stop' total_duration=3349551877 load_duration=1239653739 prompt_eval_count=48 prompt_eval_duration=236146988 eval_count=285 eval_duration=1865818464 message=Message(role='assistant', content='<think>\nOkay, let\'s see. The user wants to know the sentiment of the review: "Der Arzt war sehr freundlich und kompetent. Die Wartezeit war kurz." \n\nFirst, I need to break down the review into two sentences. The first part says the doctor was very friendly and competent. The second part mentions the wait time was short. \n\nNow, sentiment analysis. Sentiment here refers to the overall tone or emotion. Let\'s look at each part. The doctor being friendly and competent sounds positive. Then, the wait time being short is a positive aspect too. Both parts are positive. \n\nI should check if there\'s any negative or neutral part. There\'s no mention of problems or negatives here. Both sentences are good and positiv

<font color='red'>**TASK: Improve above prompt so that it outputs only one word being either `positive` or `negative`. Think about better instructions (e.g. more precise), possible additional [parameters](https://github.com/ollama/ollama/blob/main/docs/api.md#parameters) or maybe even different [models](https://ollama.com/search) or [model sizes](https://ollama.com/library/qwen3).**</font>

Links that might be helpful:

- [LLM settings](https://www.promptingguide.ai/de/introduction/settings)
- [Elements of a Prompt](https://www.promptingguide.ai/de/introduction/elements)
- [General Prompting Guidelines](https://www.promptingguide.ai/de/introduction/tips)
- [Enable/disable thinking](https://github.com/ollama/ollama-python/blob/main/examples/thinking.py) in Ollama's client

In [7]:
from ollama import Client

MODEL_TEXT = ...

client = Client(
    host = OLLAMA_HOST
)

def predict_sensitivity(text):
    
    response = client.chat(
        ...
    )
    
    return ...

In [8]:
%%time

text_pos = "Der Arzt war sehr freundlich und kompetent. Die Wartezeit war kurz."

response_pos = predict_sensitivity(text_pos)
print(response_pos, "\n\n")

text_neg = "Der Arzt war sehr unfreundlich und unkompetent. Die Wartezeit war lang."

response_neg = predict_sensitivity(text_neg)
print(response_neg, "\n\n")

positive 


negative 


CPU times: user 4.38 ms, sys: 1.05 ms, total: 5.43 ms
Wall time: 1.81 s


Let's get a "feeling" how good this works on our doctor reviews...

In [9]:
from fhnw.nlp.utils.storage import load_dataframe
from fhnw.nlp.utils.storage import download

download("https://drive.switch.ch/index.php/s/0hE8wO4FbfGIJld/download", "data/german_doctor_reviews_tokenized.parq")
data = load_dataframe("data/german_doctor_reviews_tokenized.parq")

# remove all neutral sentimens
data = data.loc[(data["label"] != "neutral")]
data.shape

(331197, 9)

In [10]:
data.head(3)

text_original  rating  \
0  Ich bin franzose und bin seit ein paar Wochen ...     2.0   
1  Dieser Arzt ist das unmöglichste was mir in me...     6.0   
2  Hatte akute Beschwerden am Rücken. Herr Magura...     1.0   

                                                text     label  sentiment  \
0  Ich bin franzose und bin seit ein paar Wochen ...  positive          1   
1  Dieser Arzt ist das unmöglichste was mir in me...  negative         -1   
2  Hatte akute Beschwerden am Rücken. Herr Magura...  positive          1   

                                         token_clean  \
0  [ich, bin, franzose, und, bin, seit, ein, paar...   
1  [dieser, arzt, ist, das, unmöglichste, was, mi...   
2  [hatte, akute, beschwerden, am, rücken, ., her...   

                                          text_clean  \
0  ich bin franzose und bin seit ein paar wochen ...   
1  dieser arzt ist das unmöglichste was mir in me...   
2  hatte akute beschwerden am rücken . herr magur...   

                                          token_stem  \
0  [franzos, seit, paar, woch, muench, zahn, schm...   
1  [arzt, unmog, leb, je, begegnet, unfreund, her...   
2  [akut, beschwerd, ruck, magura, erst, arzt, wi...   

                               token_clean_stopwords  
0  [franzose, seit, paar, wochen, muenchen, zahn,...  
1  [arzt, unmöglichste, leben, je, begegnet, unfr...  
2  [akute, beschwerden, rücken, magura, erste, ar...

In [11]:
%%time

for index, row in data[:50].iterrows():
    result = predict_sensitivity(row["text_clean"])
    print(index, result, row["label"])

0 positive positive
1 negative negative
2 positive positive
3 positive positive
4 positive positive
5 positive positive
6 positive positive
7 positive positive
8 positive positive
9 positive positive
10 negative positive
11 positive positive
12 positive positive
13 negative negative
14 positive positive
15 positive positive
16 positive positive
17 positive positive
18 positive positive
19 negative negative
20 positive positive
21 positive positive
22 negative negative
23 positive positive
24 positive positive
25 positive positive
26 positive positive
27 positive positive
28 positive positive
29 positive positive
30 positive positive
31 negative negative
32 positive positive
33 positive positive
34 positive positive
35 positive positive
36 positive positive
37 positive positive
38 positive positive
39 positive positive
40 positive positive
41 positive positive
43 positive positive
44 positive positive
45 negative negative
46 positive positive
47 positive positive
48 positive positive
49

### Some Considerations

In total it would take about (7.96s / 50 * 331187 / (60 * 60)) ~14.6 hours to classify the complete dataset.

In this respect, it might be worth the consideration to train a "simple" classifier (as we did in an earlier notebook) given training data is available. This results in way less waiting time (and CO2 emissions).

Here is the code to predict the sensitivity for the test dataset (should "only" take about 2.9 hours)...

In [12]:
from fhnw.nlp.utils.params import get_train_test_split

data_train, data_test = get_train_test_split({}, data)
data_train, data_val = get_train_test_split({}, data_train)

In [13]:
data_test.shape

(66240, 9)

In [17]:
%%time
from datetime import datetime

every_n_elements = 1000
predictions = []
count = 0

for index, row in data_test.iterrows():
    prediction = predict_sensitivity(row["text_clean"])
    predictions.append(prediction)

    count+=1

    if count % every_n_elements == 0:
        print(datetime.now().time(), "prediction:", prediction, "label:", row["label"], "total predictions", len(predictions), "of", data_test.shape[0])

18:18:25.889825 prediction: positive label: positive total predictions 1000 of 66238
18:22:33.939356 prediction: positive label: positive total predictions 2000 of 66238
18:26:45.578507 prediction: positive label: positive total predictions 3000 of 66238
18:30:54.351001 prediction: positive label: positive total predictions 4000 of 66238
18:35:07.541708 prediction: positive label: positive total predictions 5000 of 66238
18:39:15.693960 prediction: positive label: positive total predictions 6000 of 66238
18:43:23.938780 prediction: positive label: positive total predictions 7000 of 66238
18:47:34.442752 prediction: negative label: negative total predictions 8000 of 66238
18:51:43.241336 prediction: positive label: positive total predictions 9000 of 66238
18:55:50.850350 prediction: positive label: positive total predictions 10000 of 66238
18:59:59.676031 prediction: positive label: positive total predictions 11000 of 66238
19:04:06.585307 prediction: positive label: positive total pred

In [18]:
from fhnw.nlp.utils.ploting import report_classification_results

report_classification_results({}, data_test["label"].to_list(), predictions)

Set Population: 66238
Accuracy: 0.9811


              precision    recall  f1-score   support

    negative       0.86      0.97      0.91      6604
    positive       1.00      0.98      0.99     59634

    accuracy                           0.98     66238
   macro avg       0.93      0.98      0.95     66238
weighted avg       0.98      0.98      0.98     66238



In [39]:
from IPython import display
from base64 import b64decode

base64_confusion_matrix = "..."
display.Image(b64decode(base64_confusion_matrix))